In [ ]:
!nvidia-smi

Thu Mar 14 05:29:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
! pip install datasets transformers rouge-score nltk py7zr
! pip install wandb -qq

     |████████████████████████████████| 225kB 8.3MB/s 
     |████████████████████████████████| 2.3MB 48.0MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 245kB 47.2MB/s 
     |████████████████████████████████| 112kB 49.8MB/s 
     |████████████████████████████████| 901kB 31.9MB/s 
     |████████████████████████████████| 3.3MB 47.6MB/s 
     |████████████████████████████████| 358kB 36.2MB/s 
     |████████████████████████████████| 1.9MB 48.3MB/s 
     |████████████████████████████████| 2.2MB 31.6MB/s 
     |████████████████████████████████| 122kB 56.8MB/s 
ERROR: transformers 4.6.1 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.9 which is incompatible.
     |████████████████████████████████| 1.8MB 8.0MB/s 
     |████████████████████████████████| 102kB 13.7MB/s 
     |████████████████████████████████| 174kB 39.2MB/s 
     |████████████████████████████████| 133kB 36.4MB/s 
     |███████████████████████████████

In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
%env WANDB_PROJECT=bart_large_xsum_samsum

env: WANDB_PROJECT=bart_large_xsum_samsum


In [ ]:
model_checkpoint = "facebook/bart-large-xsum"

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/309 [00:00<?, ?B/s]

In [ ]:
batch_size = 4
args = Seq2SeqTrainingArguments(
    "test-dialogue-summarization",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    report_to="wandb",  # enable logging to W&B
    run_name="bart-large-xsum-samsum"  # name of the W&B run (optional)
)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import nltk
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# trainer.train()

wandb: Currently logged in as: lidiya (use `wandb login --relogin` to force relogin)


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.384400,1.481299,53.114800,28.340200,43.631000,48.439400,26.657700
1,1.045900,1.428414,54.377800,29.504400,44.564400,49.751000,28.954800
2,0.855600,1.478123,54.392100,29.807800,45.154300,49.942000,30.028100


TrainOutput(global_step=5523, training_loss=1.1156969921713653, metrics={'train_runtime': 7266.8301, 'train_samples_per_second': 0.76, 'total_flos': 144216851742720.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 961540096, 'init_mem_gpu_alloc_delta': 1625362944, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1326395392, 'train_mem_gpu_alloc_delta': 4891765248, 'train_mem_cpu_peaked_delta': 1353306112, 'train_mem_gpu_peaked_delta': 5291012608})

In [ ]:
trainer.evaluate()

{'epoch': 3.0,
 'eval_gen_len': 30.0281,
 'eval_loss': 1.478122591972351,
 'eval_mem_cpu_alloc_delta': 303104,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 1703727616,
 'eval_rouge1': 54.3921,
 'eval_rouge2': 29.8078,
 'eval_rougeL': 45.1543,
 'eval_rougeLsum': 49.942,
 'eval_runtime': 504.7343,
 'eval_samples_per_second': 1.621}

In [ ]:
wandb.finish()

train/loss,0.8556
train/learning_rate,0.0
train/epoch,3.0
train/global_step,5523
_runtime,7772
_timestamp,1622061938
_step,15
eval/loss,1.47812
eval/rouge1,54.3921
eval/rouge2,29.8078
eval/rougeL,45.1543


train/loss,█▇▆▅▃▃▃▂▁▁▁
train/learning_rate,█▇▇▆▅▄▄▃▂▂▁
train/epoch,▁▂▂▃▃▄▄▅▅▆▇▇████
train/global_step,▁▂▂▃▃▄▄▅▅▆▇▇████
_runtime,▁▂▂▃▃▃▄▄▅▅▆▆▇███
_timestamp,▁▂▂▃▃▃▄▄▅▅▆▆▇███
_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
eval/loss,█▁██
eval/rouge1,▁███
eval/rouge2,▁▇██
eval/rougeL,▁▅██


[Uploaded the model](https://huggingface.co/transformers/model_sharing.html) to the [🤗 Model Hub](https://huggingface.co/models). You can use it to generate results as shown below.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Introduction to Embedded Systems.pdf to Introduction to Embedded Systems.pdf


In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.2 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")
conversation = '''Introduction to Embedded Systems
Embedded S ystem Introduction :
We live in an era where pervasive  (spreading)  computing exists everywhere, right
from a small handheld device such as a mobile phone to the electronic control units within
automobiles or avionics. Today, large volumes of information is getting processed and
communicated over the Internet every micros econd. Buzz words such as Cloud Computing,
Big Data Mining and Internet of Things are everywhere.

'''

summarizer(conversation)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'summary_text': 'An introduction to Embedded Systems.'}]

In [ ]:
import pickle
file_path = "/content/drive/My Drive/summarizer_model.pkl"
with open(file_path, "wb") as f:
    pickle.dump(summarizer, f)

In [ ]:
print(type(conversation))

<class 'str'>


In [ ]:
len(conversation)

2256

In [ ]:
from transformers import pipeline
import textwrap

# Initialize the summarization pipeline
summarizer = pipeline("summarization", model="lidiya/bart-large-xsum-samsum")

# Conversation text
conversation = '''Introduction to Embedded Systems
Embedded S ystem Introduction :
We live in an era where pervasive  (spreading)  computing exists everywhere, right
from a small handheld device such as a mobile phone to the electronic control units within
automobiles or avionics. Today, large volumes of information is getting processed and
communicated over the Internet every micros econd. Buzz words such as Cloud Computing,
Big Data Mining and Internet of Things are everywhere.
There are two broad classifications of computing systems - general purpose
computing system and embedded computing systems. If we define these in simple words,
general purpose computing systems are those used in desktop or laptop computers, which can
process several different applications. An embedded system refers to any device that has
some computational intelligence in it. It is generally used as a stan dalone system that
repeatedly performs a specific task or as part of a large system to perform multiple tasks with
the requi red hardware and software embedded within. Systems used in printers, washing
machines, mp3 players;  CT scan machines etc. are great examples of embedded systems.
An embedded system is a constrained system and its design goals vary from a general
purpose system. The constraints are: high performance, low power consumption, small size
and low cost of the system.
The basic components of a n embedded system include hardware, software and some
mechanical  parts. Embedded hardware includes a processing unit, block of memory and I/O
sub-unit which are  called as the system resources. The embedded software can be thought of
as the application soft ware  in a small computing system or both the system and the
application software in case of a large  complex system. The system software mentioned here
is the real time operating system (RTOS) used  to manage the usage of system resources by
application soft ware.
What is an Embedded System?
 An embedded system is a system that has software embedded into computer -
hardware, which makes a system dedicated for an application or specific part of an
application or product or part of a larger system.
 An embedded syst em is designed to do a specific /few task  only'''
import textwrap

# Split the conversation into words
words = long_text.split()

# Initialize a list to store the segments
segments = []

# Initialize a variable to keep track of the current segment
current_segment = ""

segment_summaries =[]
current_lenCount=[]
# Iterate over the words in the conversation
for word in words:
    # Add the word to the current segment
    current_segment+=word+" "
    current_lenCount.append(word)


    # Check if the current segment has approximately 100 words
    if len(current_lenCount) >= 100:
        # Join the words in the current segment into a string and add it to the segments list
        # print(current_segment,'\n')
        segment_summary = summarizer(current_segment)
        segment_summaries.append(segment_summary[0]["summary_text"])
        # segments.append(" ".join(current_segment))

        # Reset the current segment
        current_segment = ""
        current_lenCount=[]


full_summary = " ".join(segment_summaries)
print(full_summary)

#1039 word OUTPUT
#4255  INPUT

An introduction to general purpose computing system and embedded computing systems. An embedded system is a device with hardware and software embedded within it. The constraints of an embedded system are: high performance, low power consumption, small size and low cost. An embedded system is a system that has software embedded into computer hardware. The host computer is a general purpose computer. The application program developed runs on the host computer rather than on the embedded system itself. The development platform is used to create the output binary image. It contains many development tools. Once the program has been written, compiled, assembled and linked, it is moved to the target platform. The target platform is executed on the target hardware platform. It consists of two entities - the The list of electronic devices is as follows:. There are many different types of embedded systems.  In embedded systems, the application program is saved in the ROM. The processing unit acc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
model = pickle.load(open('summarizer_model.pkl','rb'))


In [ ]:
words = full_summary.split()


segments = []

current_segment = ""

segment_summaries =[]
current_lenCount=[]

for word in words:

    current_segment+=word+" "
    current_lenCount.append(word)



    if len(current_lenCount) >= 100:

        segment_summary = summarizer(current_segment)
        segment_summaries.append(segment_summary[0]["summary_text"])
        # segments.append(" ".join(current_segment))

        # Reset the current segment
        current_segment = ""
        current_lenCount=[]


S2 = " ".join(segment_summaries)
print(S2)

# print(S2)


An introduction to general purpose computing system and embedded computing systems. The target platform is executed on the target hardware platform. In embedded systems, the application program is saved in the ROM. The processing unit accesses the ROM to fetch instructions sequentially and executes them within the CPU. The RAM holds the variables declared in the program, the stack and intermediate data. I/O PORTS: To provide digital communication with the outside world. D ata buses: To link these subsystems to transfer data and instructions. Timers are a fundamental c oncept in embedded systems. No. 1 contains only CPU, RAM, ROM, I/O Ports, Serial Port, Timers, General Purpose Single Purpose Microcontroller. Field programmable gate arrays are used in embedded systems to enhance memory. RAM is memory that can be accessed directly when power is removed. It is faster than ROM but expensive. Non-volatile memory is used for program and constant data. MSP430 microcontrollers use flash memory

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pypdf
from pypdf import PdfReader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 3.7 MB/s eta 0:00:00


In [ ]:
 reader = PdfReader('/content/drive/MyDrive/Introduction to Embedded Systems.pdf')

In [ ]:
print(len(reader.pages))

14


In [ ]:
page= reader.pages[0]
# long_text = print(page.extract_text())

UNIT -I 
Introduction to Embedded Systems  
Embedded S ystem Introduction : 
We live in an era where pervasive  (spreading)  computing exists everywhere, right 
from a small handheld device such as a mobile phone to the electronic control units within 
automobiles or avionics. Today, large volumes of information is getting processed and 
communicated over the Internet every micros econd. Buzz words such as Cloud Computing, 
Big Data Mining and Internet of Things are everywhere.  
There are two broad classifications of computing systems - general purpose 
computing system and embedded computing systems. If we define these in simple words, 
general purpose computing systems are those used in desktop or laptop computers, which can 
process several different applications. An embedded system refers to any device that has 
some computational intelligence in it. It is generally used as a stan dalone system that 
repeatedly performs a specific task or as part of a large system to perform multi

In [ ]:
print(type(long_text))

<class 'NoneType'>


In [ ]:
long_text=""
for i in range(len(reader.pages)):
  page = reader.pages[i]
  long_text+=(page.extract_text())

print(long_text)
# for i in page.images:
#   with open(i.name,'wb') as f:
#     f.write(i.data)

UNIT -I 
Introduction to Embedded Systems  
Embedded S ystem Introduction : 
We live in an era where pervasive  (spreading)  computing exists everywhere, right 
from a small handheld device such as a mobile phone to the electronic control units within 
automobiles or avionics. Today, large volumes of information is getting processed and 
communicated over the Internet every micros econd. Buzz words such as Cloud Computing, 
Big Data Mining and Internet of Things are everywhere.  
There are two broad classifications of computing systems - general purpose 
computing system and embedded computing systems. If we define these in simple words, 
general purpose computing systems are those used in desktop or laptop computers, which can 
process several different applications. An embedded system refers to any device that has 
some computational intelligence in it. It is generally used as a stan dalone system that 
repeatedly performs a specific task or as part of a large system to perform multi

In [ ]:
cd /content/drive/MyDrive/Hack-Quest

/content/drive/MyDrive/Hack-Quest
